In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [1]:
from proj1_helpers import *
from functions import *
y, tX, ids = load_csv_data("data/train.csv")
y[np.where(y == -1)] = 0
tX[np.where(tX == -999)] = np.nan

## Do your thing crazy machine learning thing here :) ...

In [2]:
tx = median_imputation(tX)
tx = rearrange_continuous_categorical_features(tx)
tx = build_poly(tx, 5, pairwise_interaction=False, intercept=False)
tx, mean_, std_ = gaussian_scaling(tx)
tx, todot = eigen(tx)
x_train, x_validation, y_train, y_validation = split_data(y, tx, 0.8, seed=1)

In [ ]:
newton_loss, newton_w, newton_grad_norm = logistic_newton_descent(y_train,
                                                                  x_train,
                                                                  w=np.zeros(x_train.shape[1]),
                                                                  lambda_=0.5,
                                                                  max_iters=100,
                                                                  eps=1e-6,
                                                                  w_start_OLS=True)

In [ ]:
thresh = threshold(y_train, sigmoid(x_train@newton_w))
pred = sigmoid(x_train@newton_w)
pred = (pred>thresh)*1
print(f"In sample performance : {1 - sum(np.abs(pred - y_train))/len(y_train)}")

In [ ]:
pred = sigmoid(x_validation@newton_w)
pred = (pred>thresh)*1
print(f"Out of sample performance : {1 - sum(np.abs(pred - y_validation))/len(y_validation)}")

In [26]:
newton_grad_norm

0.02318274462619573

# Recompute on the full dataset

In [11]:
y, tX, ids = load_csv_data("data/train.csv")
y[np.where(y == -1)] = 0
tX[np.where(tX == -999)] = np.nan
tx = median_imputation(tX)
tx = rearrange_continuous_categorical_features(tx)
tx = build_poly(tx, 5, pairwise_interaction=True, intercept=False)
tx, mean_, std_ = gaussian_scaling(tx)
tx, todot = eigen(tx)

[autoreload of functions failed: Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/opt/anaconda3/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/opt/anaconda3/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/kylian/Documents/GitHub/HBP/scripts/functions.py",

KeyboardInterrupt: 

In [28]:
newton_loss, newton_w, newton_grad_norm = logistic_newton_descent(y_train,
                                                                  x_train,
                                                                  w=np.zeros(x_train.shape[1]),
                                                                  max_iters=100,
                                                                  eps=1e-6,
                                                                  w_start_OLS=True)

Maximum progress reached until divergence.                        


C:\Users\Antek\GitHub\HBP\scripts\functions.py:187: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-x))
C:\Users\Antek\GitHub\HBP\scripts\functions.py:194: RuntimeWarning: divide by zero encountered in log
  (1 - y).T.dot(np.log(1 - proba)))


In [54]:
GD_loss, GD_w, GD_grad_norm = logistic_gradient_descent(y,
                                                        tx,
                                                        w=GD_w,
                                                        max_iters=10000,
                                                        gamma=0.05,
                                                        eps=1e-4,
                                                        w_start_OLS=False)

max_iters reached.                        


C:\Users\Antek\GitHub\HBP\scripts\functions.py:194: RuntimeWarning: divide by zero encountered in log
  (1 - y).T.dot(np.log(1 - proba)))


In [55]:
thresh = threshold(y, sigmoid(tx@GD_w), step=0.001)
pred = sigmoid(tx@GD_w)
pred = (pred>thresh)*1
print(f"In sample performance : {1 - sum(np.abs(pred - y))/len(y)}")

In sample performance : 0.829132


# Ready to publish

In [ ]:
tx = median_imputation(tX)
tx = rearrange_continuous_categorical_features(tx)
tx = build_poly(tx, 5, pairwise_interaction=True, intercept=False)
tx, mean_, std_ = gaussian_scaling(tx)
tx, todot = eigen(tx)

In [46]:
_, x_test, ids_test = load_csv_data("data/test.csv")
x_test[np.where(x_test == -999)] = np.nan
x_test = median_imputation(x_test)
x_test = rearrange_continuous_categorical_features(x_test)
x_test = build_poly(x_test, 5, pairwise_interaction=True, intercept=False)
x_test = (x_test - mean_) / std_
x_test = x_test.dot(todot)

In [56]:
#create prediction
y_pred = sigmoid(x_test@GD_w)
y_pred = (y_pred>thresh)*1

In [57]:
y_pred[np.where(y_pred == 0)] = -1
print(len(y_pred[np.where(y_pred == 1)])/len(y_pred[np.where(y_pred == -1)]))

0.4626987845121832


In [58]:
create_csv_submission(ids_test, y_pred, "anthony_submission_06_10_8.csv")

In [ ]:
GD_loss, GD_w, GD_grad_norm = logistic_gradient_descent(y,
                                                        tx,
                                                        w=np.zeros(tx.shape[1]),
                                                        max_iters=100,
                                                        gamma=0.5,
                                                        eps=1e-4,
                                                        w_start_OLS=True)

In [ ]:
SGD_loss, SGD_w, SGD_grad_norm = logistic_stochastic_gradient_descent(y,
                                                                      tx,
                                                                      w=np.zeros(tx.shape[1]),
                                                                      max_iters=100,
                                                                      gamma=0.5,
                                                                      batch_size=1,
                                                                      w_start_OLS=True)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)